# Tree with state propagation

In [1]:
%matplotlib widget
import bmcs_utils.api as bu
import traits.api as tr
from bmcs_utils.model import Model

In [2]:
scd = False

Each model can be regarded as a tree of model components.
Let us define models `TopModel`, `InterimModel` and `SubModel`.

In [3]:
class SubModel(Model):
    name = 'submodel'
    
    length = bu.Float(2, GEO=True)

    ipw_view = bu.View(
        bu.Item('length')
    )

In [4]:
sm = SubModel()
sm.state_change_debug = True

In [5]:
sm.length = 3

value_changed <__main__.SubModel object at 0x7f572da9a270> TraitChangeEvent(object=<__main__.SubModel object at 0x7f572da9a270>, name='length', old=2, new=3.0)


Let us define a more complex model

In [6]:
class InterimModel(Model):
    sm = bu.Instance(SubModel,())
    
    stiffness = tr.Property(bu.Float, depends_on='state_changed')
    @tr.cached_property
    def _get_stiffness(self):
        return self.sm.length * 10
    
    depends_on = ['sm']
    
    ipw_view = bu.View(
        bu.Item('stiffness', readonly=True)
    )

In [7]:
im = InterimModel(sm=sm)
im.state_change_debug = True

In [8]:
im.sm.length = 8

value_changed <__main__.SubModel object at 0x7f572da9a270> TraitChangeEvent(object=<__main__.SubModel object at 0x7f572da9a270>, name='length', old=3.0, new=8.0)
value_changed <__main__.InterimModel object at 0x7f572e7950e0> Notification from child <__main__.SubModel object at 0x7f572da9a270>


In [9]:
class TopLevelModelEagerGraphChange(Model):
    name = 'tm'
    
    sm = bu.Instance(SubModel, ())
    def _sm_default(self):
        sm = SubModel()
        self.set_sm(sm)
        return sm
    def _sm_changed(self):
        self.set_sm(self.sm)
    def set_sm(self, sm):
        self.im.sm = sm
        self.im2.sm = sm
        
    im = bu.Instance(InterimModel, ())
    im2 = bu.Instance(InterimModel, ())

    time = bu.Float(400, ALG=True)

    tree = ['sm', 'im', 'im2']


In [10]:
tl = TopLevelModelEagerGraphChange()
tl.state_change_debug = True

In [11]:
tl.sm = SubModel()
tl.sm, tl.im2.sm, tl.im.sm

graph_changed <__main__.InterimModel object at 0x7f572d76a0e0> Notification from child <__main__.SubModel object at 0x7f572d6f6d60>
graph_changed <__main__.TopLevelModelEagerGraphChange object at 0x7f572d792bd0> Notification from child <__main__.InterimModel object at 0x7f572d76a0e0>
graph_changed <__main__.InterimModel object at 0x7f572d76eea0> Notification from child <__main__.SubModel object at 0x7f572d6f6d60>
graph_changed <__main__.TopLevelModelEagerGraphChange object at 0x7f572d792bd0> Notification from child <__main__.InterimModel object at 0x7f572d76eea0>
deleting %s <__main__.SubModel object at 0x7f572d75fa90>
graph_changed <__main__.TopLevelModelEagerGraphChange object at 0x7f572d792bd0> Notification from child <__main__.SubModel object at 0x7f572d6f6d60>


(<__main__.SubModel at 0x7f572d6f6d60>,
 <__main__.SubModel at 0x7f572d6f6d60>)

In [12]:
tl.im.parents, tl.im2.parents, tl.sm.parents

({<__main__.TopLevelModelEagerGraphChange at 0x7f572d792bd0>},
 {<__main__.TopLevelModelEagerGraphChange at 0x7f572d792bd0>},
 {<__main__.InterimModel at 0x7f572d76a0e0>,
  <__main__.TopLevelModelEagerGraphChange at 0x7f572d792bd0>})

In [13]:
tl.im.sm.length = 10

value_changed <__main__.SubModel object at 0x7f572d6f6d60> TraitChangeEvent(object=<__main__.SubModel object at 0x7f572d6f6d60>, name='length', old=2, new=10.0)
value_changed <__main__.InterimModel object at 0x7f572d76eea0> Notification from child <__main__.SubModel object at 0x7f572d6f6d60>
value_changed <__main__.TopLevelModelEagerGraphChange object at 0x7f572d792bd0> Notification from child <__main__.InterimModel object at 0x7f572d76eea0>
value_changed <__main__.TopLevelModelEagerGraphChange object at 0x7f572d792bd0> Notification from child <__main__.SubModel object at 0x7f572d6f6d60>
value_changed <__main__.InterimModel object at 0x7f572d76a0e0> Notification from child <__main__.SubModel object at 0x7f572d6f6d60>
value_changed <__main__.TopLevelModelEagerGraphChange object at 0x7f572d792bd0> Notification from child <__main__.InterimModel object at 0x7f572d76a0e0>


In [14]:
tl.im, tl.im2, tl.im.sm.parents

(<__main__.InterimModel at 0x7f572d76a0e0>,
 {<__main__.InterimModel at 0x7f572d76a0e0>,
  <__main__.TopLevelModelEagerGraphChange at 0x7f572d792bd0>})

In [15]:
tl.interact()

In [16]:
im.sm.length

8.0

In [17]:
class TopLevelModelLazyGraphChange(Model):
    name = 'tm'
    
    sm = bu.Instance(SubModel, ())
        
    im = tr.Property(bu.Instance(InterimModel), depends_on='sm')
    @tr.cached_property
    def _get_im(self):
        print('im')
        return InterimModel(sm=self.sm)

    im2 = tr.Property(bu.Instance(InterimModel), depends_on='sm')
    @tr.cached_property
    def _get_im2(self):
        print('im2')
        return InterimModel(sm=self.sm)

    time = bu.Float(400, ALG=True)

    tree = ['sm', 'im', 'im2']

In [19]:
tl = TopLevelModelEagerGraphChange()
tl.sm.parents, tl.children, tl.im.children

({<__main__.InterimModel at 0x7f5724af6950>,
  <__main__.TopLevelModelEagerGraphChange at 0x7f572d9d3c70>},
 ['sm', 'im', 'im2'],
 ['sm'])

In [20]:
tl.interact()

Add change the component and check if the changes of stiffness in InterimModel get updated

In [21]:
tl.sm = SubModel()
tl.sm.length = 30
tl.im.stiffness

300.0